In [11]:
import os
import numpy as np
import torch
import random
from torchvision import transforms
from normalize import Normalize, MapToRange
from torch.utils.data import Dataset, DataLoader

from torch import nn
from torch.nn import *

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
torch.set_num_threads(1)

print(torch.cuda.is_available())

False


**Preprocessing the dataset**

In [12]:
class TrajectoryDataset(Dataset):
    
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, index):
        i, j = self.indices[index]        
        X = torch.tensor([
            self.dataset['dx'][i, j],
            self.dataset['dy'][i, j],
            self.dataset['dz'][i, j],
            self.dataset['vx'][i, j],
            self.dataset['vy'][i, j],
            self.dataset['vz'][i, j],
            self.dataset['phi'][i, j],
            self.dataset['theta'][i, j],
            self.dataset['psi'][i, j],
            self.dataset['p'][i, j],
            self.dataset['q'][i, j],
            self.dataset['r'][i, j],
            self.dataset['omega'][i, j, 0],
            self.dataset['omega'][i, j, 1],
            self.dataset['omega'][i, j, 2],
            self.dataset['omega'][i, j, 3],
#             self.dataset['Mx_ext'][i],
#             self.dataset['My_ext'][i],
#             self.dataset['Mz_ext'][i]
        ], dtype=torch.float32)
        
        U = torch.tensor([
            self.dataset['u'][i, j, 0],
            self.dataset['u'][i, j, 1],
            self.dataset['u'][i, j, 2],
            self.dataset['u'][i, j, 3]
        ], dtype=torch.float32)
        
        return X, U
    
# trajectories containing 199 points
dataset_path = 'datasets/HOVER_TO_HOVER_NOMINAL.npz'

dataset = dict()
print('loading dataset...')
# See all keys and shapes
# with np.load(dataset_path) as data:
#     for key in data.files:
#         print(key, data[key].shape)

with np.load(dataset_path) as full_dataset:
    # total number of trajectories
    num = len(full_dataset['dx'])
    print(num, 'trajectories')
    dataset = {key: full_dataset[key] for key in [
        't', 'dx', 'dy', 'dz', 'vx', 'vy', 'vz', 'phi', 'theta', 'psi', 'p', 'q', 'r','omega', 'u', 'omega_min','omega_max', 'k_omega', 'Mx_ext', 'My_ext', 'Mz_ext'
    ]}

# train/test split
batchsize_train = 256
batchsize_val = 256
train_trajectories = range(int(0.8*num))
test_trajectories = list(set(range(num)) - set(train_trajectories))

train_indices = [(i, j) for i in train_trajectories for j in range(199)]
train_set = TrajectoryDataset(dataset, train_indices)
train_loader = DataLoader(train_set, batch_size=batchsize_train, shuffle=True, num_workers=1)

test_indices = [(i, j) for i in test_trajectories for j in range(199)]
test_set = TrajectoryDataset(dataset, test_indices)
test_loader = DataLoader(test_set, batch_size=batchsize_val, shuffle=True, num_workers=1)

print('ready')

print('Amount of testing trajectories: ',len(test_trajectories),f'(Batchsize: {batchsize_val})')
print('Amount of training trajectories: ',len(train_trajectories),f'(Batchsize: {batchsize_train})')

loading dataset...
1000 trajectories
ready
Amount of testing trajectories:  200 (Batchsize: 256)
Amount of training trajectories:  800 (Batchsize: 256)


In [13]:
print(len(test_trajectories))
print(len(train_trajectories))

print(dataset['omega_min'])
print(dataset['omega_max'])

200
800
[5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500
 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 5500 550

**Calculating mean and standard deviation for normalization**

In [14]:
from tqdm import tqdm
X_mean = torch.zeros(16)
X_std = torch.zeros(16)

N=10000

for i, data in tqdm(enumerate(test_set)):
    X = data[0]
    X_mean += X
    if i>=N:
        break
X_mean = X_mean/N

print('mean:')
print(X_mean)
    
for i, data in tqdm(enumerate(test_set)):
    X = data[0]
    X_std += (X-X_mean)**2
    if i>=N:
        break

X_std = torch.sqrt(X_std/N)
print('std:')
print(X_std)

10000it [00:00, 88392.51it/s]


mean:
tensor([ 4.1395e-03, -8.1129e-04, -4.9017e-04, -8.1245e-05, -1.9836e-04,
        -3.7257e-05, -7.9109e-04,  3.7702e-04, -1.3955e-03,  9.9611e-05,
         5.9921e-04,  1.5186e-03,  5.5157e+03,  5.5167e+03,  5.5142e+03,
         5.5139e+03])


10000it [00:00, 79314.10it/s]

std:
tensor([2.1303e-01, 2.4059e-01, 1.2807e-01, 2.1328e-02, 2.5213e-02, 2.1781e-02,
        3.3380e-02, 3.0199e-02, 1.3165e-01, 4.7762e-02, 4.8536e-02, 4.8305e-02,
        1.8519e+02, 1.8685e+02, 1.6681e+02, 1.6441e+02])


**Defining the neural network**

In [15]:
class CustomSigmoid(nn.Module):
    def forward(self, x):
        return torch.clamp((-1 / 2048) * x**2 + 0.25 * x + 32, 0.0, 1.0)
    
class PiecewiseSigmoid(nn.Module):
    def forward(self, x):
        return torch.where(x < -2.5, torch.zeros_like(x),
               torch.where(x > 2.5, torch.ones_like(x),
               0.2 * x + 0.5))
class TanhSigmoid(nn.Module):
    def forward(self, x):
        return 0.5 * (torch.tanh(x / 2) + 1)
    

class HardSigmoid(nn.Module):
    def forward(self, x):
        return torch.clamp(0.2 * x + 0.5, min=0.0, max=1.0)

class FastSigmoid(nn.Module):
    def forward(self, x):
        return x / (1 + torch.abs(x))

model = nn.Sequential(
    Normalize(mean=X_mean, std=X_std),
    nn.Linear(16, 120),
    nn.ReLU(),
    nn.Linear(120, 120),
    nn.ReLU(),
    nn.Linear(120, 120),
    nn.ReLU(),
    nn.Linear(120, 4),
    nn.Sigmoid()
)
model

/home/aelarrassi/drone/normalize.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.mean = torch.tensor(mean, dtype=torch.float32)
/home/aelarrassi/drone/normalize.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.std = torch.tensor(std, dtype=torch.float32)


Sequential(
  (0): Normalize()
  (1): Linear(in_features=16, out_features=120, bias=True)
  (2): ReLU()
  (3): Linear(in_features=120, out_features=120, bias=True)
  (4): ReLU()
  (5): Linear(in_features=120, out_features=120, bias=True)
  (6): ReLU()
  (7): Linear(in_features=120, out_features=4, bias=True)
  (8): Sigmoid()
)

**Testing**

In [16]:
x1 = torch.randn(16)
print(model(x1))
# print([param.shape for param in model.parameters()])

tensor([0.6702, 0.4503, 0.7526, 0.5665], grad_fn=<SigmoidBackward0>)


**Define a Loss function and optimizer**

In [17]:
criterion = torch.nn.MSELoss()

# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
learning_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=1,  threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)


**Training loop**

In [ ]:
from tqdm import tqdm
import time
import copy

loss_list = []
loss_val_list = []
best_loss = 0.1
first = True
start_time = time.time()

# loop over the dataset multiple times
num_epochs = 100

nn_model_name = f"{dataset_path[9:-4]}_{batchsize_train}_{batchsize_val}_{learning_rate}_{num_epochs}"

for epoch in range(num_epochs):
    
    if first:
        time_remaining = '-'
    else:
        time_estimate = epoch_time*(num_epochs-epoch+1)
        if time_estimate > 60:
            if time_estimate > 3600:
                time_remaining = str(round(time_estimate/3600,2))+' h'
            else:
                time_remaining = str(round(time_estimate/60,2))+' min'
        else:
            time_remaining = str(round(time_estimate,0))+' s'
        
    first = False
    print(f"Epoch {epoch+1}/{num_epochs}, Current learning rate: {optimizer.state_dict()['param_groups'][0]['lr']}, Time remaining: {time_remaining}")

    start_time_epoch = time.time()
    
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    
    for i, (data, targets) in loop:
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(data)
        
        # Loss
        loss = criterion(outputs, targets)
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Update progressbar
        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())
        loss_list.append(loss.item())

    # Validate
    with torch.no_grad():
        # Get a random batch from the test dataset
        data_val, targets_val = next(iter(test_loader))

        # Forward pass
        outputs_val = model(data_val)

        # Loss
        loss_val = criterion(outputs_val, targets_val)

        if loss_val < best_loss:
            # Save best model
            best_model = copy.deepcopy(model)
            
            # Backup
            torch.save(model, 'neural_networks/tmp_benchmark.pt')
            
            best_loss = loss_val
            print("Best model updated!")

        # Scheduler (reduce learning rate if loss stagnates)
        scheduler.step(loss_val)
        
        loss_val_list.append(loss_val.item())

    print(f'loss = {loss:.8f}, loss validation = {loss_val:.8f} '+r' (control error: +/-'+str(round(100*np.sqrt(float(loss_val)),2))+'%)\n')

    epoch_time = (time.time() - start_time_epoch)

    loop.close()
    
# Compute excecution time
execution_time = (time.time() - start_time)    
print(f"Total training time: {round(execution_time,2)}s")

# Save best model and copy for maptorange network
torch.save(best_model, f'neural_networks/{nn_model_name}.pt')
best_model_for_maptorange = torch.load('neural_networks/tmp_benchmark.pt', weights_only=False)
print(best_model_for_maptorange)

Epoch 1/100, Current learning rate: 0.0001, Time remaining: -


Best model updated!
loss = 0.00000113, loss validation = 0.00000124  (control error: +/-0.11%)

Epoch 2/100, Current learning rate: 0.0001, Time remaining: 7.27 min


loss = 0.00000244, loss validation = 0.00000189  (control error: +/-0.14%)

Epoch 3/100, Current learning rate: 0.0001, Time remaining: 9.39 min


Best model updated!
loss = 0.00000285, loss validation = 0.00000001  (control error: +/-0.01%)

Epoch 4/100, Current learning rate: 0.0001, Time remaining: 8.48 min


loss = 0.00000124, loss validation = 0.00000177  (control error: +/-0.13%)

Epoch 5/100, Current learning rate: 0.0001, Time remaining: 7.41 min


...

loss = 0.00000000, loss validation = 0.00000000  (control error: +/-0.0%)

Epoch 85/100, Current learning rate: 1.824800363140075e-06, Time remaining: 1.23 min


loss = 0.00000000, loss validation = 0.00000062  (control error: +/-0.08%)

Epoch 86/100, Current learning rate: 1.824800363140075e-06, Time remaining: 1.2 min


loss = 0.00000000, loss validation = 0.00000057  (control error: +/-0.08%)

Epoch 87/100, Current learning rate: 1.6423203268260674e-06, Time remaining: 1.56 min


loss = 0.00000000, loss validation = 0.00000001  (control error: +/-0.01%)

Epoch 88/100, Current learning rate: 1.6423203268260674e-06, Time remaining: 1.3 min


Epoch [88/100]:  34%|██████████▉                     | 212/622 [00:02<00:05, 74.98it/s, loss=2.08e-9]

In [5]:
import matplotlib.pyplot as plt

# Compute running average (optional: smoother curve)
def running_mean(x, N=100):
    return np.convolve(x, np.ones(N)/N, mode='valid')

# Plotting loss curves
plt.figure(figsize=(12, 6))
plt.plot(loss_val_list, label='Validation Loss (per batch)', color='blue', alpha=0.3)
print(loss_val_list)
# plt.plot(running_mean(loss_list, N=100), label='Training Loss (smoothed)', color='blue')
# print(loss_list)
# plt.plot(
#     np.linspace(0, len(loss_list), len(loss_val_list)), 
#     loss_val_list, label='Validation Loss (per epoch)', color='orange'
# )

plt.xlabel('Training Iterations (batches)')
plt.ylabel('Loss')
plt.title('Training & Validation Loss over Time')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
plt.savefig('training.png')


NameError: name 'loss_val_list' is not defined

<Figure size 1200x600 with 0 Axes>

**Testing performance**

In [4]:
from tqdm import tqdm

loader = test_loader
# loop over the test dataset
loop = tqdm(enumerate(loader), total=len(loader), leave=False)
running_loss = 0

for i, (data, targets) in loop:
    outputs = model(data)
    loss = criterion(outputs, targets)
    
    running_loss += loss.item()
    
    # update progressbar
    loop.set_postfix(loss=loss.item())

loop.close()
print('average loss =', running_loss/len(loader))

NameError: name 'test_loader' is not defined

**Saving model**

In [34]:
torch.save(model, 'neural_networks/HOVER_TO_HOVER_NOMINAL.pt')
torch.save({'network_state_dict': model.state_dict()}, 'neural_networks/Drone_customsigmoid.pt')

In [3]:
model = torch.load('neural_networks/HOVER_TO_HOVER_NOMINAL.pt', weights_only=False)

In [40]:
model_processed_output =nn.Sequential(
    *model,
    MapToRange(dataset['omega_min'], dataset['omega_max'])
)

print(dataset)
torch.save(model_processed_output, 'neural_networks/HOVER_TO_HOVER_NOMINAL_.pt')

{'t': array([[0.        , 0.01010101, 0.02020202, ..., 1.97979798, 1.98989899,
        2.        ],
       [0.        , 0.01010101, 0.02020202, ..., 1.97979798, 1.98989899,
        2.        ],
       [0.        , 0.01010101, 0.02020202, ..., 1.97979798, 1.98989899,
        2.        ],
       ...,
       [0.        , 0.01010101, 0.02020202, ..., 1.97979798, 1.98989899,
        2.        ],
       [0.        , 0.01010101, 0.02020202, ..., 1.97979798, 1.98989899,
        2.        ],
       [0.        , 0.01010101, 0.02020202, ..., 1.97979798, 1.98989899,
        2.        ]]), 'dx': array([[-2.40801061, -1.20715474,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.13752423,  1.0618441 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.09140051,  0.05229611,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.01808148, -0.51022048,  0.        , ...,  0.        ,
         0.        ,  0.        ]

In [3]:
import torch

# Load your trained model
model = torch.load('neural_networks/tmp_benchmark.pt', weights_only=False)
print(model[0])

# Apply dynamic quantization (works on Linear, LSTM)
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
print(quantized_model)
# Save quantized model
torch.save({'network_state_dict': model.state_dict()}, 'neural_networks/tmp_benchmark_quantized1.pt')

Normalize()
Sequential(
  (0): Normalize()
  (1): DynamicQuantizedLinear(in_features=16, out_features=120, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (2): ReLU()
  (3): DynamicQuantizedLinear(in_features=120, out_features=120, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (4): ReLU()
  (5): DynamicQuantizedLinear(in_features=120, out_features=120, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (6): ReLU()
  (7): DynamicQuantizedLinear(in_features=120, out_features=4, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (8): Sigmoid()
)


In [1]:
import torch
from Redbit.nn import QuantLinear  # RedBit layer
from copy import deepcopy

# Load your original model
model = torch.load('neural_networks/tmp_benchmark.pt', weights_only=False)
model.eval()

# Replace all Linear layers with QuantLinear
def replace_linear_with_quant(module):
    for name, child in module.named_children():
        if isinstance(child, torch.nn.Linear):
            quant_layer = QuantLinear(
                in_features=child.in_features,
                out_features=child.out_features,
                bias=child.bias is not None,
                bit=8  # <-- 8-bit quantization
            )
            quant_layer.weight.data = deepcopy(child.weight.data)
            if child.bias is not None:
                quant_layer.bias.data = deepcopy(child.bias.data)
            setattr(module, name, quant_layer)
        else:
            replace_linear_with_quant(child)

replace_linear_with_quant(model)

# Save the modified model
torch.save(model, 'neural_networks/tmp_benchmark_redbit_quantized.pt')
load

# Optional: print the quantized model
print(model)


ModuleNotFoundError: No module named 'Redbit'

In [1]:
from tqdm import tqdm

# Use quantized model
model = quantized_model
model.eval()

loader = test_loader
running_loss = 0

loop = tqdm(enumerate(loader), total=len(loader), leave=False)

for i, (data, targets) in loop:
    data = data.cpu()      # ensure inputs are on CPU
    targets = targets.cpu()

    outputs = model(data)
    loss = criterion(outputs, targets)

    running_loss += loss.item()

    loop.set_postfix(loss=loss.item())

loop.close()
print('average loss =', running_loss / len(loader))


NameError: name 'quantized_model' is not defined

In [18]:
# Hamming distance
import torch
import torch.nn as nn
import collections
import csv

# Load the quantized model
model = torch.load('neural_networks/tmp_benchmark_quantized.pt', weights_only=False)
model.eval()

# Hamming distance between 8-bit integers
def hamming_distance(a, b=0):
    return bin(a ^ b).count("1")

# Initialize total counter for all layers (using absolute values)
global_counter = collections.Counter()
global_total = 0

# Aggregate all quantized weights (absolute values) across all layers
for name, module in model.named_modules():
    if isinstance(module, nn.quantized.dynamic.Linear):
        weight = module._packed_params._weight_bias()[0]
        flat = weight.int_repr().view(-1).cpu().numpy()
        flat_abs = abs(flat)  # take absolute values
        global_counter.update(flat_abs)
        global_total += flat_abs.size

print(f"\nTotal quantized weights (absolute values) in model: {global_total}")

# Output file
output_csv = "quantized_weight_analysis_global_abs.csv"
with open(output_csv, mode="w", newline="") as file:
    writer = csv.writer(file)            
    writer.writerow(["Quantized Value (abs)", "Count", "Percentage", "Hamming Distance", "Hamming x Percentage"])

    # Values from 0 to 127 since we're using absolute values
    for val in range(0, 128):
        count = global_counter.get(val, 0)
        percentage = (count / global_total) * 100 if global_total > 0 else 0
        hamming = hamming_distance(val & 0xFF, 0)  # 8-bit representation of abs value
        weighted_hamming = hamming * percentage

        writer.writerow([
            val,
            count,
            f"{percentage:.2f}",
            hamming,
            f"{weighted_hamming:.2f}"
        ])

print(f"\nGlobal quantized weight distribution (absolute values) saved to: {output_csv}")



Total quantized weights (absolute values) in model: 31200

Global quantized weight distribution (absolute values) saved to: quantized_weight_analysis_global_abs.csv


In [14]:

import sys
sys.path.append('/home/RedBit/QNN/')  # Adjust path as needed
import torch.nn as nn
from tools import *

class QuantizedModel(nn.Module):
    def __init__(self, wbits=4, abits=4):
        super(QuantizedModel, self).__init__()
        self.abits = abits
        self.wbits = wbits

        if self.abits == 32:
            self.act = nn.ReLU(inplace=True)
        else:
            self.act = nn.Hardtanh(inplace=True)

        self.model = nn.Sequential(
            # Normalize(mean=X_mean, std=X_std),

            QLinear(abits=self.abits, wbits=self.wbits, in_features=16, out_features=120, bias=False),
            nn.BatchNorm1d(120, eps=1e-4, momentum=0.1, affine=True),
            self.act,

            QLinear(abits=self.abits, wbits=self.wbits, in_features=120, out_features=120, bias=False),
            nn.BatchNorm1d(120, eps=1e-4, momentum=0.1, affine=True),
            self.act,

            QLinear(abits=self.abits, wbits=self.wbits, in_features=120, out_features=120, bias=False),
            nn.BatchNorm1d(120, eps=1e-4, momentum=0.1, affine=True),
            self.act,

            QLinear(abits=self.abits, wbits=self.wbits, in_features=120, out_features=4, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [3]:
import sys
# sys.path.insert(0,'/home/RedBit/QNN/models/')  # Adjust path as needed
# sys.path.remove('/home/RedBit/Baseline/tools/')
# Remove wrong path
wrong_path = '/home/RedBit/Baseline/tools'
if wrong_path in sys.path:
    sys.path.remove(wrong_path)

# Force correct path at the front
correct_path = '/home/RedBit/QNN/'
if correct_path in sys.path:
    sys.path.remove(correct_path)
sys.path.insert(0, correct_path)

# Clear cached wrong module
if 'tools' in sys.modules:
    del sys.modules['tools']
print(sys.path)
from tools import quantization
# from quantization import QLinear
# from RedBit.quantizer import quantize_pytorch_model
import torch



# Load model
model = torch.load('neural_networks/tmp_benchmark.pt' , weights_only=False)

# Quantize to 4-bit weights and activations
quantized_model = QuantizedModel(model)

# Save
torch.save(quantized_model, 'neural_networks/tmp_benchmark_quantized_int4.pt')

['/home/RedBit/QNN/', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/aelarrassi/environments/drone_env/lib/python3.10/site-packages', '/tmp/tmplsdnbq2l']


NameError: name 'QuantizedModel' is not defined

In [2]:
# from quantizer import quantize_pytorch_model
import torch
# Load quantized model
quantized_model = torch.load('neural_networks/tmp_benchmark_quantized_int4.pt', weights_only=False)
quantized_model.eval()

# Your existing test loop
from tqdm import tqdm

loader = test_loader
loop = tqdm(enumerate(loader), total=len(loader), leave=False)
running_loss = 0

for i, (data, targets) in loop:
    outputs = quantized_model(data)
    loss = criterion(outputs, targets)
    running_loss += loss.item()
    loop.set_postfix(loss=loss.item())

loop.close()
print('average loss =', running_loss / len(loader))


AttributeError: Can't get attribute 'QuantizedModel' on <module '__main__'>